## PySpark UDF (User Defined Function)

In PySpark, you can create a function and wrap it with PySpark SQL `udf()` or register it as UDF and use it on DataFrame and SQl respectively.
UDF’s are the most expensive operations hence use them only you have no choice and when essential.

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count, udf

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark UDF (User Defined Function)').getOrCreate()

#### Create a DataFrame

In [0]:
columns = ['seqno','name']

data = [
  ('1', 'john jones'),
  ('2', 'tracey smith'),
  ('3', 'amy sanders')
]

df = spark.createDataFrame(data=data,schema=columns)

df.show()

+-----+------------+
seqno| name|
+-----+------------+
 1| john jones|
 2|tracey smith|
 3| amy sanders|
+-----+------------+

#### Create a Python Function

In [0]:
def convertCase(str):
  resStr = ''
  arr = str.split(' ')
  for x in arr:
     resStr = f'{resStr}{x[0:1].upper()}{x[1:len(x)]} '
  return resStr.strip()

#### Convert a Python function to PySpark UDF

Now convert this function `convertCase()` to UDF by passing the function to PySpark SQL `udf()`.  
This function is available at `org.apache.spark.sql.functions.udf` package. Make sure you import this package before using it.

In [0]:
# Converting function to UDF
convertUDF = udf(lambda z: convertCase(z),StringType())

In [0]:
# Converting function to UDF StringType() is by default hence not required
convertUDF = udf(lambda z: convertCase(z))

#### Using UDF with DataFrame | select()

In [0]:
df.select(
  col('seqno'),
  convertUDF(col('name')).alias('name')
).show()

+-----+------------+
seqno| name|
+-----+------------+
 1| John Jones|
 2|Tracey Smith|
 3| Amy Sanders|
+-----+------------+

#### Using UDF with DataFrame | withColumn()

In [0]:
df.withColumn('fixed_name', convertUDF(col('name'))).show()

+-----+------------+------------+
seqno| name| fixed_name|
+-----+------------+------------+
 1| john jones| John Jones|
 2|tracey smith|Tracey Smith|
 3| amy sanders| Amy Sanders|
+-----+------------+------------+

#### Registering PySpark UDF & use it on SQL

In [0]:
spark.udf.register('convertUDF', convertCase, StringType())

df.createOrReplaceTempView('seq_tbl')

spark.sql('select seqno, convertUDF(name) as name from seq_tbl').show()

+-----+------------+
seqno| name|
+-----+------------+
 1| John Jones|
 2|Tracey Smith|
 3| Amy Sanders|
+-----+------------+

#### Creating UDF using annotation

In [0]:
@udf(returnType = StringType()) 
def upperCase(str):
  return str.upper()

df.withColumn('CAPS name', upperCase(col('name'))).show()

+-----+------------+------------+
seqno| name| CAPS name|
+-----+------------+------------+
 1| john jones| JOHN JONES|
 2|tracey smith|TRACEY SMITH|
 3| amy sanders| AMY SANDERS|
+-----+------------+------------+

#### Special Handling

PySpark/Spark does not guarantee the order of evaluation of subexpressions meaning expressions are not guarantee to evaluated left-to-right or in any other fixed order.  
PySpark reorders the execution for query optimization and planning hence, AND, OR, WHERE and HAVING expression will have side effects.  
When you are designing and using UDF, you have to be very careful especially with null handling as these results runtime exceptions.

##### Execution order

In [0]:
# No guarantee Name is not null will execute first
# If convertUDF(Name) like '%John%' execute first then 
# you will get runtime error

spark.sql("""
select seqno, 
       convertUDF(name) as name 
from seq_tbl 
where name is not null and convertUDF(name) like '%John%'
""").show()  

+-----+----------+
seqno| name|
+-----+----------+
 1|John Jones|
+-----+----------+

##### Handling null check

In [0]:
columns = ['seqno','name']

data = [
  ('1', 'john jones'),
  ('2', 'tracey smith'),
  ('3', 'amy sanders'),
  ('4', None)
]

df2 = spark.createDataFrame(data=data,schema=columns)

df2.show()

+-----+------------+
seqno| name|
+-----+------------+
 1| john jones|
 2|tracey smith|
 3| amy sanders|
 4| null|
+-----+------------+

In [0]:
df2.createOrReplaceTempView('seq_tbl2')
spark.sql("select convertUDF(name) from seq_tbl2").show()

--------------------------------------------------------------------------- 
 PythonException Traceback (most recent call last)
 <command-4297364997129045> in <module> 
 1 df2 . createOrReplaceTempView ( 'seq_tbl2' ) 
 ----> 2 spark . sql ( "select convertUDF(name) from seq_tbl2" ) . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 441 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 442 else : 
 --> 443 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 
 444 
 445 def __repr__ ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 PythonException : An exception was thrown from a UDF: 'AttributeError: 'NoneType' object has no attribute 'split'', from <command-4297364997129036>, line 3. Full traceback below:
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 654, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 646, in process
 serializer.dump_stream(out_iter, outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 231, in dump_stream
 self.serializer.dump_stream(self._batched(iterator), stream)
 File "/databricks/spark/python/pyspark/serializers.py", line 145, in dump_stream
 for obj in iterator:
 File "/databricks/spark/python/pyspark/serializers.py", line 220, in _batched
 for item in iterator:
 File "/databricks/spark/python/pyspark/worker.py", line 467, in mapper
 result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
 File "/databricks/spark/python/pyspark/worker.py", line 467, in <genexpr>
 result = tuple(f(*[a[o] for o in arg_offsets]) for (arg_offsets, f) in udfs)
 File "/databricks/spark/python/pyspark/worker.py", line 91, in <lambda>
 return lambda *a: f(*a)
 File "/databricks/spark/python/pyspark/util.py", line 109, in wrapper
 return f(*args, **kwargs)
 File "<command-4297364997129036>", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'

It is always best practice to check for `null` inside a UDF function rather than checking for `null` outside.  
In any case, if you can’t do a null check in UDF at lease use `IF` or `CASE WHEN` to check for `null` and call UDF conditionally.

In [0]:
spark.udf.register('_nullsafeUDF', lambda str: convertCase(str) if not str is None else '' , StringType())

Out[15]: <function __main__.<lambda>(str)>

In [0]:
spark.sql('select _nullsafeUDF(name) from seq_tbl2').show()

+------------------+
_nullsafeUDF(name)|
+------------------+
 John Jones|
 Tracey Smith|
 Amy Sanders|
 |
+------------------+

In [0]:
spark.sql("""select Seqno, _nullsafeUDF(name) as name from seq_tbl2 where name is not null and _nullsafeUDF(name) like '%John%'""").show()

+-----+----------+
Seqno| name|
+-----+----------+
 1|John Jones|
+-----+----------+

#### The end of the notebook